In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r"mental_health.csv")
data = data.iloc[:1500]
data.head()

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1


In [3]:
import re
import unicodedata
def clean_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("utf-8", "ignore")
    text = re.sub(r'[•*|►▪●]', ' ', text)
    text = re.sub(r'[\r\n\t]', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text
data['text'] = data['text'].apply(clean_text)

In [4]:
x = data['text']
y = data['label']

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#tokenize  
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
# Sequencing
sequence  = tokenizer.texts_to_sequences(x)
#voacb_size 
vocab_size = len(tokenizer.word_index)+1
print("vocab_size :" , vocab_size)
#maximun length
max_len = max(len(seq) for seq in sequence)
print("max_len " , max_len)

vocab_size : 13450
max_len  1462


In [6]:
# Padding Sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
padding_Sequence_data = pad_sequences(sequence , maxlen=max_len)
print(padding_Sequence_data)

[[   0    0    0 ...  189  835 1106]
 [   0    0    0 ...    3  600   72]
 [   0    0    0 ... 2785  800 6184]
 ...
 [   0    0    0 ...   28 1861  153]
 [   0    0    0 ... 4968    5  121]
 [   0    0    0 ...    3   11  372]]


In [7]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test  = train_test_split(padding_Sequence_data,y,random_state=2,test_size=0.2)
print(x_train.shape , x_test.shape , y_train.shape , y_test.shape )
x_train

(1200, 1462) (300, 1462) (1200,) (300,)


array([[   0,    0,    0, ...,   58,    5,  121],
       [   0,    0,    0, ..., 5177,   84,   79],
       [   0,    0,    0, ...,   46,  137,   62],
       ...,
       [   0,    0,    0, ...,  225,   20,  754],
       [   0,    0,    0, ..., 8600,  392, 8601],
       [   0,    0,    0, ..., 1410,  109,  444]], dtype=int32)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , SimpleRNN , LSTM ,  GRU ,Bidirectional,   Dense  

In [9]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size  , output_dim = 25 ))
model.add(Bidirectional(SimpleRNN(8 , activation="relu" )))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy" , optimizer="Adam" , metrics=["accuracy"])

model.fit(padding_Sequence_data,y,batch_size=32 ,epochs=8,validation_data=(x_test , y_test) )

from sklearn.metrics import accuracy_score

# --- Predict on TRAIN data ---
y_train_pred = model.predict(padding_Sequence_data)
y_train_pred = (y_train_pred > 0.5).astype(int)
train_acc = accuracy_score(y, y_train_pred)
print("Train Accuracy:", train_acc)

# --- Predict on TEST data ---
y_test_pred = model.predict(x_test)
y_test_pred = (y_test_pred > 0.5).astype(int)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_acc)


Epoch 1/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 27s 462ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.6200 - val_loss: 0.6836
Epoch 2/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 420ms/step - accuracy: 0.6853 - loss: 0.6785 - val_accuracy: 0.8233 - val_loss: 0.6555
Epoch 3/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 437ms/step - accuracy: 0.8087 - loss: 0.6416 - val_accuracy: 0.9233 - val_loss: 0.5759
Epoch 4/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 22s 475ms/step - accuracy: 0.8507 - loss: 0.5342 - val_accuracy: 0.9600 - val_loss: 0.4203
Epoch 5/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 488ms/step - accuracy: 0.9367 - loss: 0.3569 - val_accuracy: 0.9700 - val_loss: 0.2683
Epoch 6/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 24s 505ms/step - accuracy: 0.9740 - loss: 0.2315 - val_accuracy: 0.9933 - val_loss: 0.1627
Epoch 7/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 497ms/step - accuracy: 0.9913 - loss: 0.1442 - val_accuracy: 0.9967 - val_loss: 0.1053
Epoch 8/8
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 488ms/step - accuracy: 0.9967 - loss: 0.0944 - val_accuracy: 1.

In [11]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size  , output_dim = 25 ))
model.add(Bidirectional(LSTM(8 , activation="relu" )))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy" , optimizer="Adam" , metrics=["accuracy"])

model.fit(padding_Sequence_data,y,batch_size=64 ,epochs=8,validation_data=(x_test , y_test) )

from sklearn.metrics import accuracy_score

# --- Predict on TRAIN data ---
y_train_pred = model.predict(padding_Sequence_data)
y_train_pred = (y_train_pred > 0.5).astype(int)
train_acc = accuracy_score(y, y_train_pred)
print("Train Accuracy:", train_acc)

# --- Predict on TEST data ---
y_test_pred = model.predict(x_test)
y_test_pred = (y_test_pred > 0.5).astype(int)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_acc)


Epoch 1/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 643ms/step - accuracy: 0.5127 - loss: 0.6920 - val_accuracy: 0.4867 - val_loss: 0.6890
Epoch 2/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 515ms/step - accuracy: 0.5100 - loss: 0.6833 - val_accuracy: 0.5067 - val_loss: 0.6714
Epoch 3/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 541ms/step - accuracy: 0.5253 - loss: nan - val_accuracy: 0.4600 - val_loss: nan
Epoch 4/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 490ms/step - accuracy: 0.5093 - loss: nan - val_accuracy: 0.4600 - val_loss: nan
Epoch 5/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 467ms/step - accuracy: 0.5093 - loss: nan - val_accuracy: 0.4600 - val_loss: nan
Epoch 6/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 480ms/step - accuracy: 0.5093 - loss: nan - val_accuracy: 0.4600 - val_loss: nan
Epoch 7/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 540ms/step - accuracy: 0.5093 - loss: nan - val_accuracy: 0.4600 - val_loss: nan
Epoch 8/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 517ms/step - accuracy: 0.5093 - loss: nan - val_accuracy: 0.4600 - val_loss: nan
47/47 ━━━━━━

In [12]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size  , output_dim = 25 ))
model.add(Bidirectional(SimpleRNN(8 , activation="relu" )))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy" , optimizer="Adam" , metrics=["accuracy"])

model.fit(padding_Sequence_data,y,batch_size=64 ,epochs=8,validation_data=(x_test , y_test) )

from sklearn.metrics import accuracy_score

# --- Predict on TRAIN data ---
y_train_pred = model.predict(padding_Sequence_data)
y_train_pred = (y_train_pred > 0.5).astype(int)
train_acc = accuracy_score(y, y_train_pred)
print("Train Accuracy:", train_acc)

# --- Predict on TEST data ---
y_test_pred = model.predict(x_test)
y_test_pred = (y_test_pred > 0.5).astype(int)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_acc)


Epoch 1/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 362ms/step - accuracy: 0.5147 - loss: 0.6944 - val_accuracy: 0.7167 - val_loss: 0.6844
Epoch 2/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 356ms/step - accuracy: 0.7400 - loss: 0.6801 - val_accuracy: 0.8233 - val_loss: 0.6689
Epoch 3/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 374ms/step - accuracy: 0.8193 - loss: 0.6633 - val_accuracy: 0.8700 - val_loss: 0.6449
Epoch 4/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 367ms/step - accuracy: 0.8733 - loss: 0.6345 - val_accuracy: 0.9067 - val_loss: 0.6026
Epoch 5/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 374ms/step - accuracy: 0.9013 - loss: 0.5861 - val_accuracy: 0.9433 - val_loss: 0.5336
Epoch 6/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 353ms/step - accuracy: 0.9240 - loss: 0.5122 - val_accuracy: 0.9467 - val_loss: 0.4433
Epoch 7/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 414ms/step - accuracy: 0.9380 - loss: 0.4161 - val_accuracy: 0.9600 - val_loss: 0.3399
Epoch 8/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 411ms/step - accuracy: 0.9560 - loss: 0.3156 - val_accuracy: 0.9633 